In [1]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:////home/marialejandramantillab/dmeyf.db

In [3]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('~/buckets/b1/datasets/competencia_02_crudo.csv.gz')

,Success


In [5]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_02
), clientes as (
    select distinct numero_de_cliente from competencia_02
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , case 
            when mes_0 = 1 and mes_1 = 1 and mes_2 = 1 then 'CONTINUA'
            when mes_0 = 1 and mes_1 = 1 and (mes_2 is null or mes_2 = 0) then 'BAJA+2'
            when mes_0 = 1 and (mes_1 is null or mes_1 = 0) and (mes_2 is null or mes_2 = 0) then 'BAJA+1'
            else 'OTHER'
        end
        as clase_ternaria
    from todo t    
    left join competencia_02 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente  
  , clase_ternaria
from clase_ternaria where mes_0 = 1

,Success


In [10]:
%%sql
select numero_de_cliente, foto_mes, clase_ternaria from competencia_02 where numero_de_cliente=114811302 limit 10;

,numero_de_cliente,foto_mes,clase_ternaria
0,114811302,201901,BAJA+2
1,114811302,201904,CONTINUA
2,114811302,201905,CONTINUA
3,114811302,201906,CONTINUA
4,114811302,201907,CONTINUA
5,114811302,201908,CONTINUA
6,114811302,201909,CONTINUA
7,114811302,201910,CONTINUA
8,114811302,201911,CONTINUA
9,114811302,201912,CONTINUA


In [7]:
%%sql
alter table competencia_02 add column clase_ternaria VARCHAR(10)

,Success


In [8]:
%%sql
update competencia_02
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_02.numero_de_cliente = targets.numero_de_cliente and competencia_02.foto_mes = targets.foto_mes;

,Success


In [9]:
%%sql
delete from competencia_02
where  foto_mes != '202107' and not clase_ternaria in ('CONTINUA', 'BAJA+1', 'BAJA+2');

,Success


In [11]:
%%sql
copy competencia_02 to '~/buckets/b1/datasets/competencia_02.csv.gz' (FORMAT CSV, HEADER)

,Success
